In [1]:
import datetime 
import glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


#############################
## Define ash depth levels ##
## and colormap            ##
#############################

lev_min = 1e-3
lev_max = 1e3
lev_exp = np.arange(np.log10(lev_min), np.log10(lev_max).max() + 1)
levs = np.power(10, lev_exp)

########################
## Plotting functions ##
########################

clon = 180 

land_10m = cfeature.NaturalEarthFeature(
    "physical", "land", "10m", facecolor=cfeature.COLORS["land"]
)

def plot_decor(ax):
    ax.coastlines("10m")
    # ax.add_feature(ocean_10m, zorder=0)
    ax.set_extent([172,180,-42,-34])
    ax.add_feature(land_10m, zorder=-1)
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

def plot_1x1(da, levels, cmap, cbar_label, fig_label):
    fig, ax = plt.subplots(1,1, figsize=(8,6), subplot_kw={"projection": ccrs.PlateCarree(clon)})    
    da.plot.contourf(ax=ax, levels=levels, cmap=cmap, cbar_kwargs={'label': cbar_label}, transform=ccrs.PlateCarree())                                            
    plot_decor(ax)
    fig.suptitle(fig_label)
    return fig,ax

#################
## Global vars ##
#################

density = 1300 # kg/m3 andesite for Tongariro
data_pattern = 'output/lhs_gefs_sample1_202107150000/*Tongariro*nc'



# Read files into xarray dataset

In [2]:
files = glob.glob(data_pattern)
files.sort()
print(len(files))

for x in files[:3]:
    print(x)


30
output/lhs_gefs_sample1_202107150000/21071500_Tongariro_sample1_gep01_idx000_20.0_0.01.nc
output/lhs_gefs_sample1_202107150000/21071500_Tongariro_sample1_gep02_idx001_10.0_0.01.nc
output/lhs_gefs_sample1_202107150000/21071500_Tongariro_sample1_gep03_idx002_6.5_0.01.nc


In [3]:
ds = xr.open_mfdataset(files, combine='nested', concat_dim=['idx'])

# Scale system's depth computation
ds['total_deposition'].data = ds['total_deposition'].data * 1e-10 / density * 1e3 # mm
ds

<xarray.Dataset>
Dimensions:           (lat: 1401, lon: 1401, idx: 30, time: 24)
Coordinates:
  * lat               (lat) float32 -48.0 -47.99 -47.98 ... -34.02 -34.01 -34.0
  * lon               (lon) float32 166.0 166.0 166.0 ... 180.0 180.0 180.0
  * time              (time) datetime64[ns] 2021-07-15T01:00:00 ... 2021-07-16
Dimensions without coordinates: idx
Data variables:
    total_deposition  (idx, time, lat, lon) float32 dask.array<chunksize=(1, 24, 1401, 1401), meta=np.ndarray>
    hz                (idx, time) timedelta64[ns] dask.array<chunksize=(1, 24), meta=np.ndarray>
Attributes:
    title:                   HYSPLIT v5.0.1
    Conventions:             CF-1.5
    eruption_time:           2021-07-15T00:00:00
    accumulation_period_h:   24
    volcano:                 Tongariro_sample1_gep01_idx000
    src_lat:                 -39.130001068115234
    src_lon:                 175.64199829101562
    plume_height_amsl_km:    20.0
    volume_km3:              0.01
    eruption_duration_hhmm:  0100

In [4]:
eruption_time = datetime.datetime.strptime(ds.attrs['eruption_time'], '%Y-%m-%dT%H:%M:%S')
print(eruption_time)
lag_hours = np.array([int(x.total_seconds()/3600.) for x in pd.to_datetime(ds.time.data) - eruption_time])
print(lag_hours)

2021-07-15 00:00:00
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


# Excedance prob > threshold

In [5]:
def compute_arrival_time(dain, vmin):
    if dain.dims[0] != 'time':
        print(dain.shape)
        print('bad shape. Shoud be time, lat, lon')
        raise
    nt = dain.sizes['time']
    daout = xr.where(dain > vmin, 1., 0)
    daout = daout.cumsum(dim='time')
    daout = daout.where(daout > 0)
    # Assuming hourly data
    for it in range(nt):
         daout.data[it,...] = it+2 - daout.isel(time=it).data
    return daout

def compute_exc_prob_arrival_time(dain, threshold=lev_min, arrival_min=0.01):
    da = xr.where(dain > threshold, 1, 0)
    da_prob = da.sum(dim='idx')/da.sizes['idx']
    da_prob_arrival_time = compute_arrival_time(da_prob, vmin=arrival_min)
    return(da_prob, da_prob_arrival_time)




In [6]:
thresholds = [0.01, 0.1, 1, 3, 10]
arrival_min = 0.5  # arrival of the 50% excedance prob

da_probs = []
da_prob_arrival_times = []
for thres in thresholds:
    da_prob, da_prob_arrival_time = compute_exc_prob_arrival_time(
        dain=ds['total_deposition'],
        threshold=thres,    
        arrival_min=arrival_min)
    da_probs += [da_prob]
    da_prob_arrival_times += [da_prob_arrival_time]

da_probs = xr.concat(da_probs, pd.Index(thresholds, name="threshold"))
da_prob_arrival_times = xr.concat(da_prob_arrival_times, pd.Index(thresholds, name="threshold"))

In [7]:
da_prob_arrival_times

<xarray.DataArray 'total_deposition' (threshold: 5, time: 24, lat: 1401, lon: 1401)>
dask.array<concatenate, shape=(5, 24, 1401, 1401), dtype=float64, chunksize=(1, 24, 1401, 1401), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float32 -48.0 -47.99 -47.98 -47.97 ... -34.02 -34.01 -34.0
  * lon        (lon) float32 166.0 166.0 166.0 166.0 ... 180.0 180.0 180.0 180.0
  * time       (time) datetime64[ns] 2021-07-15T01:00:00 ... 2021-07-16
  * threshold  (threshold) float64 0.01 0.1 1.0 3.0 10.0

In [8]:
thres = thresholds[0]
print(thres)
da_prob_arrival_time = da_prob_arrival_times.sel(threshold=thres)
da_prob_arrival_time

da = da_prob_arrival_time.isel(time=-1)
da

0.01


<xarray.DataArray 'total_deposition' (lat: 1401, lon: 1401)>
dask.array<getitem, shape=(1401, 1401), dtype=float64, chunksize=(1401, 1401), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float32 -48.0 -47.99 -47.98 -47.97 ... -34.02 -34.01 -34.0
  * lon        (lon) float32 166.0 166.0 166.0 166.0 ... 180.0 180.0 180.0 180.0
    time       datetime64[ns] 2021-07-16
    threshold  float64 0.01

In [ ]:
plt.pcolormesh(da.data)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6), subplot_kw={"projection": ccrs.PlateCarree(clon)})    
da.plot.contourf(ax=ax, levels=lag_hours, cmap='jet_r', cbar_kwargs={'label': 'arrival'}, transform=ccrs.PlateCarree())                                            
#     plot_decor(ax)
#     fig.suptitle(fig_label)
#     return fig,ax

In [ ]:
tic = datetime.datetime.now()
it = -1

for thres in thresholds[:1]:

    da_prob = da_probs.sel(threshold=thres)
    da_prob_arrival_time = da_prob_arrival_times.sel(threshold=thres)

    # plot arrival time
    fig, ax = plot_1x1(
        da = da_prob_arrival_time.isel(time=it),
        levels = lag_hours,
        cmap = 'jet_r',
        cbar_label = 'Arrival time of {}% probability of ash thickness > {} mm [h]'.format(int(arrival_min*100), thres),
        fig_label = 'Arrival time ({} h after eruption)'.format(lag_hours[it]),
    )
#     fileplot = fileplot_prefix + '_exc_prob{}_arrival_time_lag{:02d}.png'.format(thres, lag_hours[it])
#     print(fileplot)
#     fig.savefig(fileplot, **fig_args)    
    
    
#     # plot exc1_prob 
#     prob_levs = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
#     da = da_prob.isel(time=it)
#     da = da.where(da > prob_levs[0])    
#     fig, ax = plot_1x1(
#         da = da,
#         levels = prob_levs,
#         cmap = 'viridis',
#         cbar_label = 'Probability of ash thickness > {} mm [-]'.format(thres),
#         fig_label = 'Excedance probability of {} mm ({} h after eruption)'.format(thres, lag_hours[it]),
#     )   
#     da.plot.contour(ax=ax, levels=[0.5], colors=['red'],  transform=ccrs.PlateCarree())
#     fileplot = fileplot_prefix + '_exc1_prob{}_lag{:02d}.png'.format(thres, lag_hours[it])
#     print(fileplot)
#     fig.savefig(fileplot, **fig_args)    

#     # plot exc2_prob - match risk_matrix
#     prob_levs = [0.1, 0.5, 0.9, 1.]
#     da = da_prob.isel(time=it)
#     da = da.where(da > prob_levs[0])
#     fig, ax = plot_1x1(
#         da = da,
#         levels = prob_levs,
#         cmap = 'viridis',
#         cbar_label = 'Probability of ash thickness > {} mm [-]'.format(thres),
#         fig_label = 'Excedance probability of {} mm ({} h after eruption)'.format(thres, lag_hours[it]),
#     )    
#     da.plot.contour(ax=ax, levels=[0.5], colors=['red'],  transform=ccrs.PlateCarree())
#     fileplot = fileplot_prefix + '_exc2_prob{}_lag{:02d}.png'.format(thres, lag_hours[it])
#     print(fileplot)
#     fig.savefig(fileplot, **fig_args)    

#     toc = (datetime.datetime.now() - tic).total_seconds()
#     print('Elapsed time: {} s'.format(toc))
    
    
    

## Arrival times contours

In [ ]:
# plot arrival time with hourly contours

thres = 0.01
it = -1

da_prob_arrival_time = da_prob_arrival_times.sel(threshold=thres)

cbar_label = 'Arrival time of {}% probability of ash thickness > {} mm [h]'.format(int(arrival_min*100), thres)
print(cbar_label)

fig, ax = plt.subplots(1,1, figsize=(8,6), subplot_kw={"projection": ccrs.PlateCarree(clon)})    
da_prob_arrival_time.isel(time=it).plot.contour(ax=ax, levels=lag_hours, cmap='jet_r', 
#                  cbar_kwargs={'label': cbar_label},
                 transform=ccrs.PlateCarree())                                          
plot_decor2(ax)
fig.suptitle('Arrival time ({} h after eruption)'.format(lag_hours[it]))



In [ ]:
da_probs.sizes

In [ ]:
# plot arrival time with hourly contours

thres = 0.01
nt = da_probs.sizes['time']

da = da_probs.sel(threshold=thres)

fig, ax = plt.subplots(1,1, figsize=(8,6), subplot_kw={"projection": ccrs.PlateCarree(clon)})    

for it in range(nt):
    print(it)
    da.isel(time=it).plot.contour(ax=ax, levels=[0.5],
    transform=ccrs.PlateCarree(),
)
                                    
plot_decor2(ax)
fig.suptitle('Arrival time ({} h after eruption)'.format(lag_hours[it]))



## Hazard Matrix

In [ ]:
ashbins = [0.01, 0.1, 1, 10]
probbins = [0.1, 0.5, 0.9, 1.]

# yellow, orange, red
risk = [1,2,3]
cmap_risk = ListedColormap(["yellow", "orange", "red"])

it = -1
da_probs_it = da_probs.isel(time=it)

# yellow: everything > probbins[0]
da_risk_it = xr.where(da_probs_it.sel(threshold=ashbins[0]) > probbins[0], 1, 0)


In [ ]:
da_risk_it

In [ ]:
# orange:
tmp1 = xr.where(da_probs_it.sel(threshold=ashbins[1]) > probbins[1], 1, 0)
tmp2 = xr.where(da_probs_it.sel(threshold=ashbins[2]) > probbins[0], 1, 0)
tmp = tmp1 + tmp2
tmp = xr.where(tmp > 0, 1, 0)
da_risk_it = xr.where(tmp > 0, 2, da_risk_it)

In [ ]:
# red
tmp3 = xr.where(da_probs_it.sel(threshold=ashbins[2]) > probbins[1], 1, 0)
tmp4 = xr.where(da_probs_it.sel(threshold=ashbins[1]) > probbins[2], 1, 0)
tmp = tmp3 + tmp4
tmp = xr.where(tmp > 0, 1, 0)
da_risk_it = xr.where(tmp > 0, 3, da_risk_it)

In [ ]:
da_risk_it = da_risk_it.drop('threshold')

In [ ]:
da_risk_it.where(da_risk_it > 0).plot.contourf(levels=[0,1,2,3], colors=['yellow', 'orange', 'red'])

In [ ]:
def plot_risk_matrix(iax, ashbins, probbins, cmap):
    Z = np.array([[1, 1, 2],
                  [1, 2, 3],
                  [1, 3, 3]])
    x = np.arange(Z.shape[1] + 1)
    y = np.arange(Z.shape[0] + 1)
    iax.pcolormesh(x, y, Z, shading='flat', vmin=Z.min(), vmax=Z.max(), cmap=cmap,
                  edgecolor='k')
    iax.set_xticks([0, 1., 2., 3.])
    iax.set_xticklabels(ashbins)
    iax.set_xlabel('Ash deposition [mm]')
    iax.set_yticks([0, 1., 2., 3.])
    iax.set_yticklabels(np.array(probbins)*100.)
    iax.set_ylabel('Likelihood [\%]')
    iax.set(title='Hazard matrix')

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6), subplot_kw={"projection": ccrs.PlateCarree(clon)})    
da = da_risk_it.where(da_risk_it > 0) 
da.plot.contourf(ax=ax, 
                 levels=[0,1,2,3], cmap=cmap_risk,
                 add_colorbar=False,
                 transform=ccrs.PlateCarree())                                            
plot_decor2(ax)
ax.set_extent([170,180,-42,-34])
fig.suptitle('Likelihood of accumulated ash thickness ({} h after eruption)'.format(lag_hours[it]))

# this is an inset axes over the main axes
ins_ax = fig.add_axes([.26, .5, .12, .15])
plot_risk_matrix(ins_ax, ashbins, probbins, cmap_risk)
fileplot = fileplot_prefix + '_hazard_matrix_lag{:02d}.png'.format(lag_hours[it])
print(fileplot)
fig.savefig(fileplot, **fig_args)  
    

In [ ]:
ds['total_deposition'].isel(idx=10, time=-1).plot.contour(levels=[0.01], colors=['blue'])

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(8,6), subplot_kw={"projection": ccrs.PlateCarree(clon)})    
da = da_risk_it.where(da_risk_it > 0) 
da.plot.contourf(ax=ax, 
                 levels=[0,1,2,3], cmap=cmap_risk,
                 add_colorbar=False,
                 transform=ccrs.PlateCarree())  

ds['total_deposition'].isel(idx=0, time=-1).plot.contour(ax=ax, levels=[0.5], colors=['blue'],  transform=ccrs.PlateCarree())
    
plot_decor2(ax)
ax.set_extent([170,180,-42,-34])
fig.suptitle('Likelihood of accumulated ash thickness ({} h after eruption)'.format(lag_hours[it]))

# this is an inset axes over the main axes
ins_ax = fig.add_axes([.26, .5, .12, .15])
plot_risk_matrix(ins_ax, ashbins, probbins, cmap_risk)
fileplot = fileplot_prefix + '_hazard_matrix_lag{:02d}_v2.png'.format(lag_hours[it])
print(fileplot)
fig.savefig(fileplot, **fig_args)  
    

# Ens quantiles

In [ ]:
# to be able to use functions like quantile
print(ds.chunks)
ds = ds.chunk({'idx': ds.sizes['idx']})
print(ds.chunks)

In [ ]:
tic = datetime.datetime.now()

quantiles = [0.1, 0.5, 0.9]
dait_quantiles = ds['total_deposition'].isel(time=-1).quantile(quantiles, dim='idx')

toc = (datetime.datetime.now() - tic).total_seconds()
print('Elapsed time: {} s'.format(toc))

dait_quantiles.plot(col='quantile')

toc = (datetime.datetime.now() - tic).total_seconds()
print('Elapsed time: {} s'.format(toc))

In [ ]:
dait_quantiles.plot(col='quantile', figsize=(18,5),
    levels=levs, cmap=cmap2, norm=norm_levs, 
#     transform=ccrs.PlateCarree(),
)

In [ ]:
%time it 
tic  = datetime.datetime.now()

# Plot 3x1 common colorbar
# https://stackoverflow.com/questions/69435068/change-colorbar-limit-for-changing-scale-with-matplotlib-3-3
nq = len(quantiles)
fig, axs = plt.subplots(1,nq, figsize=(6*nq,5), subplot_kw={"projection": ccrs.PlateCarree(clon)})
for i in range(nq):
    ax = axs[i]
    da = dait_quantiles.isel(quantile=i)
    p1 = da.plot.contourf(ax=ax, levels=levs, cmap=cmap2, norm=norm_levs,
        transform=ccrs.PlateCarree(),
        add_colorbar=False)
    ax.set_title('{:.0f}th Quantile'.format(quantiles[i]*100))
    plot_decor2(ax)

fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.88, 0.15, 0.03, 0.7])
# Doesn't do the extend (deprecated matplotlib +3.3) - bad colors!
# cb = fig.colorbar(p1, cax=cbar_ax, extend='both')

cb = matplotlib.colorbar.ColorbarBase(ax=cbar_ax, 
    cmap=cmap2,
    norm=matplotlib.colors.LogNorm(vmin=levs_plus[0], vmax=levs_plus[-1]),
    # to use 'extend', you must
    # specify two extra boundaries:
    boundaries=levs_plus, #[0]+bounds+[13],
    extend='both',
#     ticks=levs, # optional
#     spacing='proportional',
)

cb.set_label('Ash thickness [mm]')

fig.suptitle('Ensemble quantiles ({} h after eruption)'.format(lag_hours[it]))

fileplot = fileplot_prefix + '_quantiles_lag{:02d}.png'.format(lag_hours[it])
print(fileplot)
fig.savefig(fileplot, **fig_args)

toc = (datetime.datetime.now() - tic).total_seconds()
print('Elapsed time: {} s'.format(toc))

In [ ]:
! ls output/lhs_gefs_sample1_202107150000/*.nc
# 202107150000/*//*/21071500_Tongariro*.nc